<a href="https://colab.research.google.com/github/tiaaburton/Milestone-II-Machine-Learning-Project/blob/main/Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 59.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=a9755b16ed33a40363fdcd7b8ac5f3c6db661dfedcee415b653ae995542b70ef
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [90]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark

from google.colab import files

import pandas as pd
import numpy as np
import re

In [ ]:
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab 6M Exploration")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
spark.newSession()

In [ ]:
schema = StructType([\
    StructField("visitorId", StringType(), True),\
    StructField("visitNumber", StringType(), True),\
    StructField("visitId", StringType(), True),\
    StructField("visitStartTime", StringType(), True),\
    StructField("date", StringType(), True),\
    StructField("totals", StringType(), True),\
    StructField("trafficSource", StringType(), True),\
    StructField("device", StringType(), True),\
    StructField("geoNetwork", StringType(), True),\
    StructField("customDimensions", StringType(), True),\
    StructField("hits", StringType(), True),\
    StructField("userId", StringType(), True),\
    StructField("clientId", StringType(), True),\
    StructField("channelGrouping", StringType(), True),\
    StructField("socialEngagementType", StringType(), True)])

In [185]:
def get_selected_columns(filename='/content/drive/Shareddrives/SIADS - 694-695 Team Drive/Notebooks/final_columns.txt'):
  """
  filename: """
  columns = []
  with open(filename, 'r') as f:
    for ln in f.readlines():
      col = (ln.strip().split('\xa0\xa0')[1])
      groups = re.search(r"'(\w.+)'", col)
      columns.append(groups.group(1))
  return columns

def read_csv(filename):
  """
  Reads csv in with spark dataframe. 
  
  Params 
    filename
  
  Output
    Pyspark.sql dataframe object with selected columns in"""
  columns = get_selected_columns()

  selection = []
  for col in columns:
    splits = col.split('.')
    if splits[0] not in selection + ['clientId']:
      selection.append(splits[0])

  data = spark.read\
  .option('header','true')\
  .option('delimiter',',')\
  .option('index','false')\
  .option('mode','DROPMALFORMED')\
  .csv(filename)

  data = data.select(selection)

  return data

def add_col_from_json(dataframe, json_col, json_field):
  """
  json_col:
  json_field: """
  fieldname = '$.'+json_field
  df = dataframe.withColumn(json_field, get_json_object(dataframe[json_col],fieldname))
  return df

In [186]:
columns = get_selected_columns('/content/drive/Shareddrives/SIADS - 694-695 Team Drive/Notebooks/final_columns.txt')
analytics_data = read_csv('/content/drive/Shareddrives/SIADS - 694-695 Team Drive/Sample Datasets/2016 - 2017 Google Analytics Dataset.csv')

drop_cols = set()

for col in columns:
  splits = col.split('.')
  if len(splits) == 2:
    analytics_data = add_col_from_json(analytics_data, splits[0], splits[1])
  elif len(splits) == 3:
    drop_cols.add(splits[0])
    drop_cols.add(splits[1])
    analytics_data = add_col_from_json(analytics_data, splits[0], splits[1])
    analytics_data = add_col_from_json(analytics_data, splits[1], splits[2])

In [187]:
analytics_data

DataFrame[totals: string, fullVisitorId: string, hits: string, channelGrouping: string, socialEngagementType: string, geoNetwork: string, trafficSource: string, device: string, visits: string, pageviews: string, timeOnSite: string, bounces: string, transactions: string, newVisits: string, screenviews: string, uniqueScreenviews: string, timeOnScreen: string, totalTransactionRevenue: string, type: string, country: string, source: string, medium: string, isTrueDirect: string, browser: string, operatingSystem: string, deviceCategory: string, hour: string, minute: string, transaction: string, item: string, dataSource: string, transactionRevenue: string, productName: string, productCategory: string, itemRevenue: string, appInfo: string, screenDepth: string, eCommerceAction: string, action_type: string]

In [ ]:
# spark.write.csv(output_path)

In [ ]:
# pyspark.sql.SparkSession.stop